# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [4]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [5]:
m=0

## Зададим начальное значение f$_{0}$

In [6]:
f=1

## Посчитаем Z

In [8]:
z=np.zeros([206,55])

In [9]:
for i in range(0,55):
    for j in range(0,206):
        zsum=0
        for k in range(0,35):
            zsum+=Xdata_numpy['X'][k,j,i]*Ydata.iloc[k,m]
        z[j,i]=zsum

In [10]:
z

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        , 30.42741355, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 29.76362119, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 29.36079377, ...,  0.        ,
         0.        ,  0.        ]])

## Разложим Z на вектора W$^{J}_{f}$ и W$^{J}_{f}$